### Exercise 2: Plan-Execute + Reflection Hybrid

**Task:** Combine Plan-Execute and Reflection patterns to create a high-quality multi-step agent.

**Concept:** Use Plan-Execute to break down complex tasks, then use Reflection on the final output to ensure quality.

**Requirements:**
1. Create a hybrid state that includes:
   - Plan-Execute state (input, plan, current_step, results)
   - Reflection state (draft, critique, iterations)
2. Flow: Planner → Executor (loop) → Generator (uses results) → Critic → Refiner (if needed) → Finalizer
3. The Generator creates initial output from all executor results
4. The Critic evaluates the complete output (not individual steps)
5. Refine the final output based on critique (max 2 reflection iterations)
6. Track both execution progress AND reflection quality

**Architecture:**
```
START
  ↓
Planner (creates plan)
  ↓
Executor (executes each step)
  ↓ (loops until all steps done)
  ↓
Generator (synthesizes results into draft)
  ↓
Critic (evaluates quality)
  ↓
Should Refine?
  ↓              ↓
Refiner       Finalizer
  ↓              ↓
Critic         END
  (loops)
```

**Test scenario:**
```
Task: "Research the benefits of Python programming, create a summary, and make it beginner-friendly"

Expected flow:
1. PLAN: 
   - Step 1: Search for Python benefits
   - Step 2: Identify key benefits
   - Step 3: Create summary
2. EXECUTE: Complete all steps
3. GENERATE: Create initial summary from results
4. REFLECT: Critique summary for beginner-friendliness
5. REFINE: Simplify language and add examples
6. FINAL: High-quality, beginner-friendly summary
```

**Deliverables:**
1. Combined state definition (TypedDict)
2. All necessary nodes (planner, executor, generator, critic, refiner, finalizer)
3. Graph with proper edges and conditional routing
4. Test with the scenario above
5. Print output showing:
   - The plan
   - Each execution step
   - Initial draft
   - Critique
   - Refined version (if applicable)
   - Final output

---

In [8]:
# Imports
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode, create_react_agent
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from IPython.display import Image, display
from typing import List, TypedDict
import operator
import os
import json


print("✅ All imports successful")

✅ All imports successful


In [2]:
# Load API key
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found!")

print("✅ API key loaded")

✅ API key loaded


In [3]:
# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    api_key=openai_api_key
)

print(f"✅ LLM initialized: {llm.model_name}")

✅ LLM initialized: gpt-4o-mini


In [9]:
class PlanExecuteReflectState(TypedDict):
    # User input
    task: str

    # Plan–Execute
    plan: List[str]
    current_step: int
    results: List[str]

    # Reflection
    draft: str
    critique: str
    iterations: int

    # Final output
    final_output: str


In [10]:
def planner(state: PlanExecuteReflectState):
    prompt = f"""
Create a clear step-by-step plan to complete the task:

Task: {state['task']}
"""
    response = llm.invoke(prompt)

    plan = response.content.strip().split("\n")

    return {
        "plan": plan,
        "current_step": 0,
        "results": []
    }


In [11]:
def executor(state: PlanExecuteReflectState):
    step = state["plan"][state["current_step"]]

    prompt = f"""
Execute the following step and return the result:

Step: {step}
"""
    response = llm.invoke(prompt)

    return {
        "results": state["results"] + [response.content],
        "current_step": state["current_step"] + 1
    }


In [12]:
def should_continue_execution(state: PlanExecuteReflectState):
    if state["current_step"] < len(state["plan"]):
        return "executor"
    return "generator"


In [13]:
def generator(state: PlanExecuteReflectState):
    combined_results = "\n".join(state["results"])

    prompt = f"""
Using the information below, write a clear and concise draft:

{combined_results}
"""
    response = llm.invoke(prompt)

    return {
        "draft": response.content,
        "iterations": 0
    }


In [14]:
def critic(state: PlanExecuteReflectState):
    prompt = f"""
Critique the following draft.
If it is good, respond ONLY with "APPROVED".
Otherwise, give clear improvement suggestions.

Draft:
{state['draft']}
"""
    response = llm.invoke(prompt)

    return {
        "critique": response.content
    }


In [15]:
def refiner(state: PlanExecuteReflectState):
    prompt = f"""
Improve the draft using the critique below.

Draft:
{state['draft']}

Critique:
{state['critique']}
"""
    response = llm.invoke(prompt)

    return {
        "draft": response.content,
        "iterations": state["iterations"] + 1
    }


In [16]:
def should_refine(state: PlanExecuteReflectState):
    if "APPROVED" in state["critique"]:
        return "finalizer"

    if state["iterations"] >= 2:
        return "finalizer"

    return "refiner"


In [17]:
def finalizer(state: PlanExecuteReflectState):
    return {
        "final_output": state["draft"]
    }


In [18]:
builder = StateGraph(PlanExecuteReflectState)

builder.add_node("planner", planner)
builder.add_node("executor", executor)
builder.add_node("generator", generator)
builder.add_node("critic", critic)
builder.add_node("refiner", refiner)
builder.add_node("finalizer", finalizer)

builder.add_edge(START, "planner")
builder.add_edge("planner", "executor")

builder.add_conditional_edges(
    "executor",
    should_continue_execution
)

builder.add_edge("generator", "critic")

builder.add_conditional_edges(
    "critic",
    should_refine
)

builder.add_edge("refiner", "critic")
builder.add_edge("finalizer", END)


In [19]:
reflection_agent = builder.compile()


In [21]:
result = reflection_agent.invoke({
    "task": "Research the benefits of Python programming, create a summary, and make it beginner-friendly"
})

print(result["final_output"])


### Benefits of Python Programming: A Beginner-Friendly Summary

Python is a versatile and powerful programming language that has gained immense popularity among beginners and experienced developers alike. Here are some key benefits of learning and using Python:

1. **Easy to Learn and Use**: 
   - Python has a simple and readable syntax that resembles plain English, making it an excellent choice for beginners. This simplicity allows new programmers to focus on learning programming concepts rather than getting bogged down by complex syntax.

2. **Versatile and Flexible**: 
   - Python can be used for a wide range of applications, from web development and data analysis to artificial intelligence and scientific computing. This versatility means that once you learn Python, you can apply your skills in various fields.

3. **Large Community and Support**: 
   - Python has a vast and active community of developers. This means that beginners can easily find resources, tutorials, and forums to